In [49]:
import zipfile
import os


def extract_zip(zip_file_path, extract_to='inference_db'):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:

        os.makedirs(extract_to, exist_ok=True)
        zip_ref.extractall(extract_to)

        return os.path.join(extract_to, zip_ref.namelist()[0])


def read_text_files(extract_to):
    for file in os.listdir(extract_to):
        with open(os.path.join(extract_to, file), 'r') as f:
            file_lines = f.readlines()
            return file_lines

In [50]:
url_zip = "/home/navneeth/EgoPro/dnn/vits_infer/test_text.zip"
print(url_zip)

#get zip file
path = extract_zip(url_zip)
print(path)
file_lines = read_text_files(f"./{path}")



/home/navneeth/EgoPro/dnn/vits_infer/test_text.zip
inference_db/test_text/


In [51]:
file_lines

['hello my name is navneeth']